<a href="https://colab.research.google.com/github/iremaricii/basic_money_laundering/blob/main/basic_money_laundering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -q kaggle

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ealtman2019/ibm-transactions-for-anti-money-laundering-aml

!unzip ibm-transactions-for-anti-money-laundering-aml.zip

In [ ]:
!mkdir dataset
!mv HI-Large_Trans.csv dataset/
!mv HI-Large_Patterns.txt dataset/
!mv HI-Medium_Patterns.txt dataset/
!mv HI-Medium_Trans.csv dataset/
!mv HI-Small_Patterns.txt dataset/
!mv HI-Small_Trans.csv dataset/
!mv LI-Medium_Patterns.txt dataset/
!mv LI-Medium_Trans.csv dataset/
!mv LI-Small_Patterns.txt dataset/
!mv LI-Small_Trans.csv dataset/

In [17]:
import pandas as pd

In [18]:
df=pd.read_csv("/content/dataset/HI-Medium_Trans.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [21]:
df.drop_duplicates(inplace=True) #DataFrame'deki tekrar eden satırları tespit edip siler.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
data = pd.read_csv("/content/dataset/HI-Medium_Trans.csv")

In [ ]:
# Pozitif örneklerin tamamını al
positive = data[data["Is Laundering"] == 1]

# Negatif örnekleri, pozitif sayısına eşitle
negative = data[data["Is Laundering"] == 0].sample(n=len(positive), random_state=42)

In [ ]:
# Yeni dengeli veri setini oluştur
balanced_data = shuffle(pd.concat([positive, negative]), random_state=42)

In [ ]:
# Sayısal olmayan sütunları çıkar
X = balanced_data.select_dtypes(include=['number']).drop('Is Laundering', axis=1)
y = balanced_data["Is Laundering"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = Sequential([
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [26]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

#Kod, modelin test verilerindeki tahminlerini 0.5 eşiğiyle ikili sınıflara çevirip performansını raporlar.

551/551 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.61      0.60      8808
           1       0.60      0.60      0.60      8807

    accuracy                           0.60     17615
   macro avg       0.60      0.60      0.60     17615
weighted avg       0.60      0.60      0.60     17615

Confusion Matrix:
 [[5331 3477]
 [3499 5308]]
